# Chapter 9: Subqueries

As seen in Chapter 3, subqueries are quite powerful tools to filter conditions.

Let's explore that some more.


In [1]:
import os

from dotenv import load_dotenv
from sqlalchemy import create_engine, URL, select, func
from sqlalchemy.orm import Session
import pandas as pd

from utils import print_sql_statement


load_dotenv()

url_object = URL.create(
    os.environ["DB_ENGINE"],
    username=os.environ["DB_USER"],
    password=os.environ["DB_PASSWD"],
    host=os.environ["DB_HOST"],
    database=os.environ["DB_NAME"],
)

engine = create_engine(url_object)

# Non-correlated, Multirow Subqueries

For simple queries like find the employees who are superiors, would be written like so using the IN operator

```sql
SELECT
    emp_id
    , fname
    , lname
    , title
FROM employee
WHERE emp_id IN (
    SELECT superior_emp_id
    FROM employee
)
```

or the less intuitive ANY and ALL operators.


```sql
SELECT
    emp_id
    , fname
    , lname
    , title
FROM employee
WHERE emp_id = ANY (
    SELECT superior_emp_id
    FROM employee
)
```

While the ANY and ALL operators are a clumsy in this case, the ALL operator would be more intuitive in following case: Find all accounts having an available balance smaller than all of Frank Tucker's accounts.

In [12]:
from sqlalchemy import and_

from model import Account, Individual


with Session(engine) as session:
    df = pd.read_sql_query(
        """
        SELECT
            account_id
            , avail_balance
        FROM account
        WHERE avail_balance < ALL (
            SELECT
                avail_balance
            FROM account a
            JOIN individual ind ON a.cust_id = ind.cust_id
            WHERE ind.fname = 'Frank' AND ind.lname = 'Tucker'
        )
        ORDER BY account_id;
        """,
        con=session.connection()
    )
    target_individual: Individual = (
        session.query(Individual)
        .where(
            and_(
                Individual.fname == "Frank",
                Individual.lname == "Tucker"
            )
        )
        .scalar()
    )
    target_accounts: list[Account] = (
        session.query(Account)
        .where(Account.cust_id == target_individual.cust_id)
        .all()
    )
    results = sorted(
        (
            (acct.account_id, acct.avail_balance)
            for acct in session.query(Account)
            if all(
                acct.avail_balance < tgt_acct.avail_balance
                for tgt_acct in target_accounts
            )
        ),
        key=lambda tup: tup[0]
    )

print(df)
print(results)

   account_id  avail_balance
0           2         500.00
1           5         200.00
2          10         534.12
3          11         767.77
4          14         122.37
5          19         387.99
6          21         125.67
7          25           0.00
[(2, 500.0), (5, 200.0), (10, 534.12), (11, 767.77), (14, 122.37), (19, 387.99), (21, 125.67), (25, 0.0)]


As a closing example, the IN operator supports multicolumn functionality.

```sql
SELECT
    account_id
    , product_cd
    , cust_id
FROM account
WHERE (open_branch_id, open_emp_id) IN (
    SELECT
        b.branch.id,
        e.emp_id
    FROM branch
    b JOIN employee e
        ON b.branch_id = e.assigned_branch_id
    WHERE (
        b.name = 'Woburn Branch'
        AND e.title LIKE '%Teller'
    )
);
```

# Correlated Subqueries

Unlike uncorrelated subqueries, correlated subqueries are dependent upon the outer scope requirements. Often the `[NOT] EXISTS` operator is used

For example, find all the accounts with transactions on '2000-01-15'.


In [28]:
from datetime import date

from model import Transaction


with Session(engine) as session:
    df = pd.read_sql_query(
        """
        SELECT
            a.account_id
            , a.product_cd
            , a.cust_id
            , a.avail_balance
        FROM
            account a
        WHERE EXISTS (
            SELECT
                1
            FROM
                transaction t
            WHERE (
                t.account_id = a.account_id
                AND CAST(t.txn_date AS DATE) = '2000-01-15'
            )
        );
        """,
        con=session.connection()
    )
    # Since Python does not have the EXISTS operator, there
    # is the bool operator that is similar. Although it is not
    # required in the bottom usage within an if-expression, I'm
    # including it for clarity between SQL and Python
    trans: Transaction
    acct: Account
    results = [
        (acct.account_id, acct.product_cd, acct.cust_id, acct.avail_balance)
        for acct in session.query(Account)
        if bool(
            set(
                True
                for trans in acct.account_transactions
                if trans.txn_date.date() == date(2000, 1, 15)
            )
        )
    ]

print(df)
print(results)

   account_id product_cd  cust_id  avail_balance
0           1        CHK        1        1057.75
1           2        SAV        1         500.00
2          11        SAV        4         767.77
[(1, 'CHK', 1, 1057.75), (2, 'SAV', 1, 500.0), (11, 'SAV', 4, 767.77)]


# Subqueries for Table Fabrication

Count the number of accounts with the following balances

 * "Small Fry": [0, 5000)
 * "Average Joe": [5000, 10000)
 * "Heavy Hitters": [10000, ∞)

In [62]:
from collections import Counter

from model import ProductType, Customer


with Session(engine) as session:
    df = pd.read_sql_query(
        """
        SELECT
            grps.name
            , COUNT(*) num_cust
        FROM
        (
            SELECT
                SUM(acct.avail_balance) cust_balance
            FROM account acct JOIN product prod
                ON acct.product_cd = prod.product_cd
            WHERE prod.product_type_cd = 'ACCOUNT'
            GROUP BY acct.cust_id
        ) cust_rollup JOIN
        (
            SELECT
                'Small Fry' name
                , 0 low_limit
                , 4999.99 high_limit
            UNION ALL
            SELECT
                'Average Joe' name
                , 5000 low_limit
                , 9999.99 high_limit
            UNION ALL
            SELECT
                'Heavy Hitter' name
                , 10000 low_limit
                , 9999999.99 high_limit
        ) grps ON cust_rollup.cust_balance BETWEEN
            grps.low_limit AND grps.high_limit
        GROUP BY grps.name
        ORDER BY grps.name;
        """,
        con=session.connection()
    )

    # Using native Python with the ORM is a little easier with querying the Customers
    # via the established relationships

    table_bounds: dict[str, tuple[float, float]] = {
        "Small Fry": (0.00, 5_000 - 0.01),
        "Average Joe": (5_000., 10_000 - 0.01),
        "Heavy Hitter": (10_000., 1E11 - 0.01),
    }
    results = Counter(
        [
            # Store the account group type
            key
            for key, (low_limit, high_limit) in table_bounds.items()
            # For all the customers, sum up their "ACCOUNT" balances
            for cust in session.query(Customer)
            if (
                low_limit <= sum(
                    acct.avail_balance
                    if acct.account_product.product_type_cd == "ACCOUNT"
                    else 0.
                    for acct in cust.customer_accounts
                ) <= high_limit
                and
                any(
                    acct2.account_product.product_type_cd == "ACCOUNT"
                    for acct2 in cust.customer_accounts
                )
            )
        ]
    )


print(df)
print(results)

           name  num_cust
0   Average Joe         2
1  Heavy Hitter         4
2     Small Fry         5
Counter({'Small Fry': 5, 'Heavy Hitter': 4, 'Average Joe': 2})


In [ ]:
[

]